In [1]:
using QuantumOptics
using NBInclude
using LinearAlgebra
using Einsum
# lattice
@nbinclude("Lattice_Basis_Neighbors_Hops.ipynb"; regex=r"#.*executeme")
# mb tools
@nbinclude("QOJulia_utils.ipynb")

proj_MB_Int (generic function with 1 method)

# Parameters

In [2]:
param_dict=Dict{}()

Dict{Any, Any}()

In [3]:
LL = [3,3]
N = LL[1]*LL[2]
vv_lattice = [ [1., 0], [0, 1.] ]
b1=[0, 0]
vv_basis = [b1]
param_dict["N"] = LL[1]*LL[2]

param_dict["LL"] = LL
param_dict["vv_lattice"] = vv_lattice
param_dict["vv_basis"] = vv_basis

nb = 1 # number of basis
t=-1 # hopping amplitude
p=1
q=3
ϕ=p/q # magnetic flux per plaquet
param_dict["gauge"] = "Landau"
param_dict["t"]=t
param_dict["ϕ"]=ϕ
θθ = [0, 0] # initial values for twist angles
param_dict["θθ"]=θθ

sites, site_coord, site_number = get_sites(LL, vv_lattice, vv_basis);
param_dict["sites"] = sites
param_dict["site_coord"] = site_coord
param_dict["site_number"] = site_number

pbc=true # periodic boundary condition
param_dict["pbc"] = pbc
neighbors = [ [1, 0, 1], [-1, 0, 1], [0, 1, 1], [0, -1, 1]  ]
neighbors_list = [ neighbors ]
param_dict["neighbors_list"] = neighbors_list

U = 0.5
param_dict["U"] = U;

In [4]:
# single ptl
basis = NLevelBasis(N) 
param_dict["basis"] = basis;

param_dict["model_H1"] = get_Hofstadter_hamiltonian

H_T = get_Hofstadter_hamiltonian(param_dict)
H1 = get_sp_op(basis, H_T)

E1, U1 = eigenstates(dense(H1));
E1

9-element Vector{Float64}:
 -2.7320508075688728
 -2.732050807568871
 -2.732050807568871
  0.7320508075688776
  0.7320508075688779
  0.7320508075688781
  1.9999999999999998
  1.9999999999999998
  2.0000000000000004

In [5]:
PN = 2
param_dict["PN"] = PN
states_mb = bosonstates(basis, PN)
basis_mb = get_basis_mb(param_dict)

ManyBody(onebodybasis=NLevel(N=9), states:45)

In [6]:
H1_mb = get_mb_op(basis_mb, H1);
param_dict["dimH_mb"] = length(basis_mb)
H_NI_mb = get_H_mb(param_dict);

In [7]:
E_NI, U_NI = eigenstates(dense(H_NI_mb));

In [8]:
E_NI[1:10]

10-element Vector{Float64}:
 -5.464101615137779
 -5.464101615137776
 -5.464101615137773
 -5.464101615137763
 -5.464101615137753
 -5.464101615137751
 -2.000000000000006
 -2.0000000000000013
 -2.0
 -1.999999999999999

In [9]:
Vint_mb = SparseOperator(basis_mb)
for m in 1:N
    Vint_mb += U/2 * number(basis_mb, m) * ( number(basis_mb, m) - identityoperator(basis_mb) ) 
end

In [10]:
m = 1
V2 = U/2 * transition(basis,m,m)⊗transition(basis,m,m)
for m in 2:N
    V2 += U/2 * transition(basis,m,m)⊗transition(basis,m,m)
end

In [11]:
Vint2_mb = manybodyoperator(basis_mb, V2);

In [12]:
isapprox( Vint_mb, Vint2_mb )

true

In [13]:
H_mb = H_NI_mb + Vint2_mb;
eigenenergies(dense(H_mb))[1:10]
# to be compared with projection results

10-element Vector{Float64}:
 -5.425763381179248
 -5.425763381179245
 -5.425763381179242
 -5.346240396489923
 -5.346240396489921
 -5.346240396489921
 -1.9292774894793527
 -1.9292774894793512
 -1.929277489479351
 -1.9188553622105469

In [24]:
# projection
substates = 1:9
basis_tilde = SubspaceBasis(basis, U1[substates])

Subspace(superbasis=NLevel(N=9), states:9)

In [25]:
P1 = projector(basis, basis_tilde)
P1_dag = dagger(P1)
P1M = P1.data
P1_dagM = P1_dag.data
P2M = (P1⊗P1).data
P2_dagM = (P1_dag⊗P1_dag).data;
H1M = H1.data;
V2M = V2.data;

In [26]:
@einsum H1_tildeM[i,j] := P1_dagM[i,k] * H1M[k,l] * P1M[l,j];
@einsum V2_tildeM[i2,j2] :=  P2_dagM[i2,k2] * V2M[k2,l2] * P2M[l2,j2];

In [27]:
basis_tilde2 = basis_tilde⊗basis_tilde

H1_tilde = Operator(basis_tilde, H1_tildeM);

V2_tilde = Operator(basis_tilde2, V2_tildeM);

basis_mb_tilde  = ManyBodyBasis(basis_tilde, states_mb)

H_NI_tilde_mb = manybodyoperator(basis_mb_tilde, H1_tilde);

Vint_tilde_mb = manybodyoperator(basis_mb_tilde, V2_tilde);

H_tilde_mb = H_NI_tilde_mb + Vint_tilde_mb;

In [28]:
eigenenergies(dense(H_tilde_mb))[1:10]
# to be compared with exact results above

┌ Warning: The given operator is not hermitian. If this is due to a numerical error make the operator hermitian first by calculating (x+dagger(x))/2 first.
└ @ QuantumOptics ~/.julia/packages/QuantumOptics/eon5P/src/spectralanalysis.jl:76


10-element Vector{ComplexF64}:
  -5.425763381179256 + 1.3802279128488492e-16im
  -5.425763381179239 - 1.1102230246251565e-16im
  -5.425763381179237 + 6.661338147750939e-16im
  -5.346240396489922 + 9.471532561791193e-17im
  -5.346240396489919 + 2.1183687617826348e-16im
  -5.346240396489913 - 1.7074408857710412e-16im
 -1.9292774894793574 - 2.706168622523819e-16im
 -1.9292774894793538 + 9.849976737030808e-17im
 -1.9292774894793536 - 2.775559473595315e-17im
 -1.9188553622105466 - 4.85722573273506e-17im